In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [ ]:
# Suppress warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')


import findspark
findspark.init()

In [3]:
# Import libraries
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler

In [4]:
# Create SparkSession
spark = SparkSession.builder.appName("Airfoil_noise").getOrCreate()

24/05/01 23:29:25 WARN Utils: Your hostname, javier-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.81.128 instead (on interface ens33)
24/05/01 23:29:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/05/01 23:29:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv

--2024-05-01 23:29:46--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60682 (59K) [text/csv]
Saving to: ‘NASA_airfoil_noise_raw.csv’

NASA_airfoil_noise_ 100%[===================>]  59,26K   256KB/s    in 0,2s    

2024-05-01 23:29:47 (256 KB/s) - ‘NASA_airfoil_noise_raw.csv’ saved [60682/60682]



In [6]:
# Load dataset
df = spark.read.csv("NASA_airfoil_noise_raw.csv", header=True, inferSchema=True)

In [7]:
df.show(5)

+---------+-------------+-----------+------------------+-----------------------+----------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevel|
+---------+-------------+-----------+------------------+-----------------------+----------+
|      800|          0.0|     0.3048|              71.3|             0.00266337|   126.201|
|     1000|          0.0|     0.3048|              71.3|             0.00266337|   125.201|
|     1250|          0.0|     0.3048|              71.3|             0.00266337|   125.951|
|     1600|          0.0|     0.3048|              71.3|             0.00266337|   127.591|
|     2000|          0.0|     0.3048|              71.3|             0.00266337|   127.461|
+---------+-------------+-----------+------------------+-----------------------+----------+
only showing top 5 rows



In [8]:
rowcount1 = df.count()
print(rowcount1)

1522


In [9]:
# Drop duplicates
df = df.dropDuplicates()

rowcount2 = df.count()
print(rowcount2)

1503


In [10]:
# Drop Na
df = df.dropna()

rowcount3 = df.count()
print(rowcount3)

1499


In [11]:
df = df.withColumnRenamed("SoundLevel","SoundLevelDecibels")

In [12]:
# Save df
df.write.parquet("NASA_airfoil_noise_cleaned.parquet")

24/05/01 23:37:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [13]:
# Evaluate
print("Part 1 - Evaluation")

print("Total rows = ", rowcount1)
print("Total rows after dropping duplicate rows = ", rowcount2)
print("Total rows after dropping duplicate rows and rows with null values = ", rowcount3)
print("New column name = ", df.columns[-1])

import os

print("NASA_airfoil_noise_cleaned.parquet exists :", os.path.isdir("NASA_airfoil_noise_cleaned.parquet"))

Part 1 - Evaluation
Total rows =  1522
Total rows after dropping duplicate rows =  1503
Total rows after dropping duplicate rows and rows with null values =  1499
New column name =  SoundLevelDecibels
NASA_airfoil_noise_cleaned.parquet exists : True


# ML Pipeline

In [14]:
df.columns

['Frequency',
 'AngleOfAttack',
 'ChordLength',
 'FreeStreamVelocity',
 'SuctionSideDisplacement',
 'SoundLevelDecibels']

In [15]:
# Assemble the input columns 'Cylinders','Engine_Disp','Horsepower','Weight','Accelerate','Year' into a single column "features"
assembler = VectorAssembler(inputCols=['Frequency','AngleOfAttack','ChordLength','FreeStreamVelocity','SuctionSideDisplacement'], outputCol="features")

In [16]:
# Scale the "features" using standard scaler and store in "scaledFeatures" column
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

In [17]:
# Create a LinearRegression stage to predict "MPG"
lr = LinearRegression(featuresCol="scaledFeatures", labelCol="SoundLevelDecibels")

In [18]:
# Pipeline
pipeline = Pipeline(stages=[assembler, scaler, lr])

In [19]:
# Split the data
(trainingData, testingData) = df.randomSplit([0.7, 0.3], seed=42)

In [20]:
# Fit the pipeline using the training data 
pipelineModel = pipeline.fit(trainingData)

24/05/01 23:46:43 WARN Instrumentation: [f02f9d84] regParam is zero, which might cause numerical instability and overfitting.
24/05/01 23:46:44 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/05/01 23:46:44 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/05/01 23:46:50 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/05/01 23:46:50 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [21]:
print("Part 2 - Evaluation")
print("Total rows = ", rowcount3)
ps = [str(x).split("_")[0] for x in pipeline.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  1499
Pipeline Stage 1 =  VectorAssembler
Pipeline Stage 2 =  StandardScaler
Pipeline Stage 3 =  LinearRegression
Label column =  SoundLevelDecibels


# Model Evaluation

In [22]:
# Make predictions
predictions = pipelineModel.transform(testingData)

In [23]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="mse")
mse = evaluator.evaluate(predictions)
print(mse)

21.66707441821937


In [24]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="mae")
mae = evaluator.evaluate(predictions)
print(mae)

3.6076987618473133


In [25]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="r2")
r2 = evaluator.evaluate(predictions)
print(r2)

0.5591898788366854


In [27]:
print("Part 3 - Evaluation")

print("Mean Squared Error = ", round(mse,2))
print("Mean Absolute Error = ", round(mae,2))
print("R Squared = ", round(r2,2))

lrModel = pipelineModel.stages[-1]

print("Intercept = ", round(lrModel.intercept,2))

Part 3 - Evaluation
Mean Squared Error =  21.67
Mean Absolute Error =  3.61
R Squared =  0.56
Intercept =  132.6


# Model Persistance

In [29]:
pipelineModel.write().save("SoundLevelDecibels")

In [31]:
loadedPipelineModel = PipelineModel.load("SoundLevelDecibels")

In [32]:
predictions = loadedPipelineModel.transform(testingData)

In [33]:
predictions.select("SoundLevelDecibels","prediction").show()

+------------------+------------------+
|SoundLevelDecibels|        prediction|
+------------------+------------------+
|           125.966|131.43112322688123|
|           121.527|122.37513428895186|
|           129.116|119.76654085222165|
|           123.514|119.04621915131851|
|           119.209|121.65469957424325|
|           118.667|120.74413436739461|
|           128.829|128.61259875575482|
|           125.951|127.03273741105095|
|           129.934| 126.9787883875116|
|           123.118|122.64687671558957|
|           133.261|130.74125122169872|
|           116.468|119.66435019649991|
|            130.94| 126.4828302672953|
|           131.467|128.53068553110066|
|           118.998|120.48664737249673|
|           121.664|121.91541156379522|
|           126.995|124.30637196522255|
|           126.616|124.49239597695033|
|           121.318|123.21368560528607|
|           125.727| 120.1047990180752|
+------------------+------------------+
only showing top 20 rows



In [34]:
print("Part 4 - Evaluation")

loadedmodel = loadedPipelineModel.stages[-1]
totalstages = len(loadedPipelineModel.stages)
inputcolumns = loadedPipelineModel.stages[0].getInputCols()

print("Number of stages in the pipeline = ", totalstages)
for i,j in zip(inputcolumns, loadedmodel.coefficients):
    print(f"Coefficient for {i} is {round(j,4)}")

Part 4 - Evaluation
Number of stages in the pipeline =  3
Coefficient for Frequency is -3.9373
Coefficient for AngleOfAttack is -2.5227
Coefficient for ChordLength is -3.294
Coefficient for FreeStreamVelocity is 1.5372
Coefficient for SuctionSideDisplacement is -1.7176


In [35]:
spark.stop()